### Khai báo thư viện

In [1]:
import pandas as pd

### Kết nối với gg drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Đọc file data.csv

In [5]:
df = pd.read_csv('/content/drive/MyDrive/data.csv')

In [7]:
df.head(10)

,URL,Content
0,https://www.facebook.com/share/p/14DqtuJaps/,"Mọi người ơiii, hãy vào link này để sở hữu bộ ..."
1,https://www.facebook.com/share/p/14DqtuJaps/,Nhạc vẫn đỉnh như ngày nào
2,https://www.facebook.com/share/p/14DqtuJaps/,Chúc mừng Sơn Tùng M-TP và MB Bank
3,https://www.facebook.com/share/p/14DqtuJaps/,Ok zai iu của mẹ
4,https://www.facebook.com/share/p/14DqtuJaps/,mê nhạc mê anhhhh
5,https://www.facebook.com/share/p/14DqtuJaps/,Xịn quáaaa
6,https://www.facebook.com/share/p/14DqtuJaps/,Tuyệt vời a iu
7,https://www.facebook.com/share/p/14DqtuJaps/,Mê a quá phải ls ạ
8,https://www.facebook.com/share/p/14DqtuJaps/,Ước gì anh tùng được làm chủ tịch nước
9,https://www.facebook.com/share/p/14DqtuJaps/,Bài hát hay tràn đầy năng lượng nha!!!


In [11]:
df.shape

(9255, 2)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9255 entries, 0 to 9254
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   URL      9255 non-null   object
 1   Content  9008 non-null   object
dtypes: object(2)
memory usage: 144.7+ KB


### Làm sạch data

Loại bỏ các khoảng trắng

In [14]:
df_cleaned = df.dropna()

In [15]:
df_cleaned.shape

(9008, 2)

Loại bỏ content trùng lặp

In [16]:
df_cleaned = df.drop_duplicates(subset=['Content'])

In [17]:
df_cleaned.shape

(8473, 2)

### Lưu lại data vừa làm sạch

In [20]:
df_cleaned.to_csv('data_cleaned.csv', index=False)